In [52]:
import pandas as pd
import numpy as np
from collections import Counter
from PMTK.sampling.preferences_sampler import *
from PMTK.pref.preferences import *
from PMTK.utility.utility_solver import *
from PMTK.sampling.subset_samplers import *
from PMTK.utility.model_solver import *
from PMTK.utility.subset_finder import *
from PMTK.sampling.gibbs import *
from PMTK.data.film_dataset import *
from PMTK.sampling.decider import *
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

def build_approx_theta(prf, init_theta = None):
    connivents = []
    if not init_theta:
        init_theta = [EMPTY_SET]
    theta = init_theta
    min_k = 1
    c  = get_connivent(theta, prf)
    cpt = 0
    while c:
        if not c in connivents:
            connivents.append(c)
        cit = get_candidate_iterator(c)
        skey = sorted(cit.keys())[0]
        b = False
        for k in cit:
            if b:
                break
            for i in cit[k]:
                s = set(i)
                for t in s:
                    b = b or (check_connivence_resolution(c, t) and not t in theta)
                    if not t in theta and check_connivence_resolution(c, t):
                        theta.append(t)
        c  = get_connivent(theta, prf)
        cpt = cpt + 1
        print("solved connivent: ", cpt, " with", theta)
    #a = additivity(theta)
    #for c_i in connivents:
    #    cit = get_candidate_iterator(c_i)
    #    for k in cit:
    #        if k > a:
    #            break
    #        for i in cit[k]:
    #            for t in i:
    #                if not t in theta and check_connivence_resolution(c_i,t):
    #                    theta.append(t)
    #
    return theta

In [53]:
fd = Film_Dataset(60,9)
prf = fd.get_preferences_items(9)
t_heuristic = build_approx_theta(prf, [EMPTY_SET])
t_heuristic

solved connivent:  1  with [(5,), (4,)]
solved connivent:  2  with [(5,), (4,), (0,), (1,)]
solved connivent:  3  with [(5,), (4,), (0,), (1,), (2,), (3,)]
solved connivent:  4  with [(5,), (4,), (0,), (1,), (2,), (3,), (7,), (13,), (6,), (12,)]
solved connivent:  5  with [(5,), (4,), (0,), (1,), (2,), (3,), (7,), (13,), (6,), (12,), (11,), (10,), (16,)]


[(5,),
 (4,),
 (0,),
 (1,),
 (2,),
 (3,),
 (7,),
 (13,),
 (6,),
 (12,),
 (11,),
 (10,),
 (16,)]

In [54]:
t_mins = get_kernels_lex3(prf,t_heuristic)
t_mins

Found n°1 =  [(5,), (1,), (2,), (7,), (13,), (11,)] with obj: 12
Found n°2 =  [(4,), (0,), (2,), (13,), (6,), (10,)] with obj: 12
Found n°3 =  [(5,), (1,), (3,), (6,), (12,), (11,)] with obj: 12
Found n°4 =  [(4,), (1,), (3,), (13,), (6,), (10,)] with obj: 12
Found n°5 =  [(5,), (1,), (3,), (13,), (6,), (10,)] with obj: 12
Found n°6 =  [(4,), (1,), (3,), (13,), (6,), (11,)] with obj: 12
Found n°7 =  [(5,), (1,), (3,), (13,), (6,), (11,)] with obj: 12
Found n°8 =  [(5,), (0,), (3,), (7,), (12,), (11,)] with obj: 12
Found n°9 =  [(4,), (0,), (3,), (7,), (12,), (11,)] with obj: 12
Found n°10 =  [(5,), (1,), (3,), (7,), (12,), (11,)] with obj: 12
Found n°11 =  [(4,), (1,), (2,), (7,), (13,), (10,)] with obj: 12
Found n°12 =  [(4,), (0,), (3,), (7,), (12,), (10,)] with obj: 12
Found n°13 =  [(4,), (0,), (2,), (7,), (13,), (10,)] with obj: 12
Found n°14 =  [(4,), (0,), (3,), (7,), (13,), (10,)] with obj: 12
Found n°15 =  [(4,), (0,), (2,), (7,), (13,), (11,)] with obj: 12
Found n°16 =  [(5,)

[[(5,), (1,), (2,), (7,), (13,), (11,)],
 [(4,), (0,), (2,), (13,), (6,), (10,)],
 [(5,), (1,), (3,), (6,), (12,), (11,)],
 [(4,), (1,), (3,), (13,), (6,), (10,)],
 [(5,), (1,), (3,), (13,), (6,), (10,)],
 [(4,), (1,), (3,), (13,), (6,), (11,)],
 [(5,), (1,), (3,), (13,), (6,), (11,)],
 [(5,), (0,), (3,), (7,), (12,), (11,)],
 [(4,), (0,), (3,), (7,), (12,), (11,)],
 [(5,), (1,), (3,), (7,), (12,), (11,)],
 [(4,), (1,), (2,), (7,), (13,), (10,)],
 [(4,), (0,), (3,), (7,), (12,), (10,)],
 [(4,), (0,), (2,), (7,), (13,), (10,)],
 [(4,), (0,), (3,), (7,), (13,), (10,)],
 [(4,), (0,), (2,), (7,), (13,), (11,)],
 [(5,), (1,), (3,), (7,), (13,), (11,)],
 [(4,), (1,), (2,), (7,), (13,), (11,)],
 [(4,), (1,), (3,), (7,), (13,), (11,)],
 [(5,), (1,), (2,), (7,), (13,), (10,)],
 [(5,), (0,), (2,), (7,), (13,), (11,)],
 [(5,), (1,), (2,), (7,), (12,), (11,)],
 [(5,), (1,), (2,), (13,), (6,), (11,)],
 [(5,), (0,), (2,), (13,), (6,), (10,)],
 [(4,), (0,), (3,), (7,), (13,), (11,)],
 [(5,), (0,), (3